In [ ]:
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from datetime import datetime
import os.path
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from PIL import Image
import glob
import sys
import cmocean.cm as cmo
import time
import matplotlib.image as image
# Bluemarble must be local. It drastically improves speed of code and also the cartopy call to bluemarble cgi is broken atm. 02/01/2022 
pngDir='/glade/u/home/jsimkins/logos/'
img = plt.imread(f'{pngDir}bluemarble.png')
img_extent = (-180, 180, -90, 90)
logo1 = image.imread(f'{pngDir}noaa.png')
logo2 = image.imread(f'{pngDir}gfdl.png')
logo3 = image.imread(f'{pngDir}rutgers.png')
logo4 = image.imread(f'{pngDir}esmg.png')

img_extent = (-180, 180, -90, 90)
#workdir = ""/Users/james/Documents/Github/esm_lab/MOM6/analysis/output/nwa25/Cheyenne/2021-11-05/"
workdir = "./"
outdir="images/"

with open(workdir + 'diag_table') as f:
    expName = f.readline()[:-1]

try:
    file_prefix = sys.argv[1]
except:
    print("Add the file we're useing as an argument!")
        
print(file_prefix)

ds = xr.open_dataset(workdir + file_prefix, engine='netcdf4')
ds=ds.isel(time=slice(0,len(ds.time.values)))
ds_grid = xr.open_dataset("/glade/work/jsimkins/gridInfo/nwa25/land_mask.nc")
var_list = ['sss','ssh050','sst']

# Note that in previous versions of this code, the for loop below was much more concise. However, there were weird issues with imshow and the bluemarble image would be distorted.
for var in var_list:
    time.sleep(5)
    img = plt.imread('/glade/work/jsimkins/bluemarble.png')
    img_extent = (-180, 180, -90, 90)
    for i in range(0,len(ds.time.values)):
        if os.path.isfile(workdir + "/images/" + var + datetime.strptime(str(ds.time[i].values), "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M") + ".png") == False:
            if var=='ssh050':
                figure = plt.figure(figsize=(12,12))
                left = 0.01
                bottom = 0.01
                width = 0.99
                height = 0.99
                ax1 = plt.axes([left, bottom, width, height], projection=ccrs.PlateCarree(), extent=[-100, -36, 5, 55], facecolor='black')
                ax1.imshow(img, origin='upper', extent=img_extent, transform=ccrs.PlateCarree(),vmin=0,vmax=255)
                im = ax1.pcolormesh(ds_grid.x.values, ds_grid.y.values, ds.ssh[i].values, cmap=cmo.balance, vmin=-0.5, vmax=0.5)
                cb = plt.colorbar(im, ticks=[-0.25,0,0.25], shrink=0.6)
                cb.set_label('Sea Surface Height (m)', color='w', fontsize=12)
                plt.title(str("MOM6 NWA25 Sea Surface Height - " + str(ds.time[i].values)), fontsize=16, color='w')
                gl = ax1.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True, x_inline=False, y_inline=False)
                gl.top_labels = False
                gl.left_labels = False
                gl.right_labels=True
                gl.bottom_labels=True
                gl.xlines = True
                gl.ylines = True
                gl.xlocator = mticker.FixedLocator([-100,-90,-80,-70, -60, -50, -40])
                gl.ylocator = mticker.FixedLocator([10, 20, 30, 40, 50])
                gl.xformatter = LONGITUDE_FORMATTER
                gl.yformatter = LATITUDE_FORMATTER
                gl.xlabel_style = {'color': 'darkgreen', 'weight': 'normal'}
                gl.ylabel_style = {'color': 'darkgreen', 'weight': 'normal'}
                cb.ax.yaxis.set_tick_params(color='w')
                plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color='w')
                ax1.imshow(logo1, origin='upper', extent=(-99, -95.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax1.imshow(logo2, origin='upper', extent=(-95, -91.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax1.imshow(logo3, origin='upper', extent=(-91, -87.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax1.imshow(logo4, origin='upper', extent=(-87, -83.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())

                outfile = str(workdir + outdir + var + datetime.strptime(str(ds.time[i].values), "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M") + ".png")
                plt.savefig(outfile, facecolor='black', bbox_inches='tight', format='png', dpi=90)
                plt.close()
            if var=='sst':
                figure = plt.figure(figsize=(12,12))
                left = 0.01
                bottom = 0.01
                width = 0.99
                height = 0.99
                ax2 = plt.axes([left, bottom, width, height], projection=ccrs.PlateCarree(), extent=[-100, -36, 5, 55], facecolor='black')
                ax2.imshow(img, origin='upper', extent=img_extent, transform=ccrs.PlateCarree(),vmin=0,vmax=255)
                im = ax2.pcolormesh(ds_grid.x.values, ds_grid.y.values, ds.sst[i].values, cmap='turbo', vmin=0, vmax=30)
                cb = plt.colorbar(im, ticks=[5,10,15,20,25], shrink=0.6)
                cb.set_label('Sea Surface Temperature (deg C))', color='w', fontsize=12)
                plt.title(str("MOM6 NWA25 Sea Surface Temperature - " + str(ds.time[i].values)), fontsize=16, color='w')
                gl = ax2.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True, x_inline=False, y_inline=False)
                gl.top_labels = False
                gl.left_labels = False
                gl.right_labels=True
                gl.bottom_labels=True
                gl.xlines = True
                gl.ylines = True
                gl.xlocator = mticker.FixedLocator([-100,-90,-80,-70, -60, -50, -40])
                gl.ylocator = mticker.FixedLocator([10, 20, 30, 40, 50])
                gl.xformatter = LONGITUDE_FORMATTER
                gl.yformatter = LATITUDE_FORMATTER
                gl.xlabel_style = {'color': 'darkgreen', 'weight': 'normal'}
                gl.ylabel_style = {'color': 'darkgreen', 'weight': 'normal'}
                cb.ax.yaxis.set_tick_params(color='w')
                plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color='w')
                ax2.imshow(logo1, origin='upper', extent=(-99, -95.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax2.imshow(logo2, origin='upper', extent=(-95, -91.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax2.imshow(logo3, origin='upper', extent=(-91, -87.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax2.imshow(logo4, origin='upper', extent=(-87, -83.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())

                outfile = str(workdir + "/images/" + var + datetime.strptime(str(ds.time[i].values), "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M") + ".png")
                plt.savefig(outfile, facecolor='black', bbox_inches='tight', format='png', dpi=90)
                plt.close()
            if var=='sss':
                figure = plt.figure(figsize=(12,12))
                left = 0.01
                bottom = 0.01
                width = 0.99
                height = 0.99
                ax3 = plt.axes([left, bottom, width, height], projection=ccrs.PlateCarree(), extent=[-100, -36, 5, 55], facecolor='black')
                ax3.imshow(img, origin='upper', extent=img_extent, transform=ccrs.PlateCarree(),vmin=0,vmax=255)
                im = ax3.pcolormesh(ds_grid.x.values, ds_grid.y.values, ds.sss[i].values, cmap=cmo.haline, vmin=30, vmax=40)
                cb = plt.colorbar(im, ticks=[32,34,36,38], shrink=0.6)
                cb.set_label('Sea Surface Salinity (psu)', color='w', fontsize=12)
                plt.title(str("MOM6 NWA25 Sea Surface Salinity - " + str(ds.time[i].values)), fontsize=16, color='w')
                gl = ax3.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True, x_inline=False, y_inline=False)
                gl.top_labels = False
                gl.left_labels = False
                gl.right_labels=True
                gl.bottom_labels=True
                gl.xlines = True
                gl.ylines = True
                gl.xlocator = mticker.FixedLocator([-100,-90,-80,-70, -60, -50, -40])
                gl.ylocator = mticker.FixedLocator([10, 20, 30, 40, 50])
                gl.xformatter = LONGITUDE_FORMATTER
                gl.yformatter = LATITUDE_FORMATTER
                gl.xlabel_style = {'color': 'darkgreen', 'weight': 'normal'}
                gl.ylabel_style = {'color': 'darkgreen', 'weight': 'normal'}
                cb.ax.yaxis.set_tick_params(color='w')
                plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color='w')
                ax3.imshow(logo1, origin='upper', extent=(-99, -95.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax3.imshow(logo2, origin='upper', extent=(-95, -91.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax3.imshow(logo3, origin='upper', extent=(-91, -87.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax3.imshow(logo4, origin='upper', extent=(-87, -83.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                outfile = str(workdir + "/images/" + var + datetime.strptime(str(ds.time[i].values), "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M") + ".png")
                plt.savefig(outfile, facecolor='black', bbox_inches='tight', format='png', dpi=90)
                plt.close()

            
    img_names = sorted(glob.glob(workdir + outdir + var + "*.png"))
    dur_vals = []
    for i in range(0,len(img_names)):
        if i != len(img_names)-1:
            dur_vals.append(200)
    dur_vals.append(3000)

    img, *imgs = [Image.open(f) for f in img_names]
    img.save(fp=workdir + "/gifs/" + expName + "_" + file_prefix[0:4] + var + '.gif', format='GIF', append_images=imgs,
         save_all=True, duration=dur_vals, loop=0)